In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import pickle 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
da=pickle.load(open('data','rb'))

In [4]:
da=da.reset_index(drop=True)

In [5]:
da

,match_id,batting_team,bowling_team,deliveries,total,player_out,city
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
76954,1432,Sri Lanka,Australia,19.3,1,0,Colombo
76955,1432,Sri Lanka,Australia,19.4,0,0,Colombo
76956,1432,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
76957,1432,Sri Lanka,Australia,19.6,2,0,Colombo


In [6]:
da=da.rename(columns={'total':'runs'})

In [7]:
da.head(2)

,match_id,batting_team,bowling_team,deliveries,runs,player_out,city
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne


In [8]:
da.groupby('match_id')['runs'].cumsum()

0          0
1          0
2          1
3          3
4          3
        ... 
76954    125
76955    125
76956    125
76957    127
76958    128
Name: runs, Length: 76959, dtype: int64

In [9]:
da['current_score']= da.groupby('match_id')['runs'].cumsum()

In [10]:
da['wickets']=np.nan

da.loc[da['player_out']==0,'wickets']=0
da.loc[da['player_out']!=0,'wickets']=1

In [11]:
da['wickets']=da['wickets'].astype('int')

In [12]:
wic=da.groupby('match_id')['wickets'].cumsum().values
da['wickets_left']=10-wic

In [13]:
da['overs']=da['deliveries'].apply(lambda x: str(x).split('.')[0])
da['balls']=da['deliveries'].apply(lambda x: str(x).split('.')[1])

In [14]:
da['overs']=da['overs'].astype('int')
da['balls']=da['balls'].astype('int')

In [15]:
da['balls_bowled']=((da['overs'])*6)+da['balls']

In [16]:
120-da['balls_bowled']

0        119
1        118
2        117
3        116
4        115
        ... 
76954      3
76955      2
76956      1
76957      0
76958     -1
Name: balls_bowled, Length: 76959, dtype: int32

In [17]:
da['balls_left']=120-da['balls_bowled']
da['balls_left']=da['balls_left'].apply(lambda x: 0 if x<0 else x)

In [18]:
da['current_run_rate']= (da['current_score']*6)/da['balls_bowled']

In [19]:
da.dtypes

match_id              int64
batting_team         object
bowling_team         object
deliveries          float64
runs                  int64
player_out           object
city                 object
current_score         int64
wickets               int32
wickets_left          int32
overs                 int32
balls                 int32
balls_bowled          int32
balls_left            int64
current_run_rate    float64
dtype: object

In [20]:
groups=da.groupby('match_id')

match_ids=da['match_id'].unique()
last_five=[]
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())

In [21]:
da['last_five_over_runs']=last_five

In [22]:
da

,match_id,batting_team,bowling_team,deliveries,runs,player_out,city,current_score,wickets,wickets_left,overs,balls,balls_bowled,balls_left,current_run_rate,last_five_over_runs
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,10,0,1,1,119,0.000000,NaN
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,10,0,2,2,118,0.000000,NaN
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,10,0,3,3,117,2.000000,NaN
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,10,0,4,4,116,4.500000,NaN
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,10,0,5,5,115,3.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76954,1432,Sri Lanka,Australia,19.3,1,0,Colombo,125,0,2,19,3,117,3,6.410256,32.0
76955,1432,Sri Lanka,Australia,19.4,0,0,Colombo,125,0,2,19,4,118,2,6.355932,32.0
76956,1432,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,1,1,19,5,119,1,6.302521,32.0
76957,1432,Sri Lanka,Australia,19.6,2,0,Colombo,127,0,1,19,6,120,0,6.350000,33.0


In [23]:
dt=da.groupby('match_id')['runs'].sum().reset_index().merge(da,on='match_id')

In [24]:
dt=dt[['batting_team','bowling_team','city','current_score','wickets_left','balls_left','current_run_rate','last_five_over_runs','runs_x']]

In [25]:
dt

,batting_team,bowling_team,city,current_score,wickets_left,balls_left,current_run_rate,last_five_over_runs,runs_x
0,Australia,Sri Lanka,Melbourne,0,10,119,0.000000,NaN,168
1,Australia,Sri Lanka,Melbourne,0,10,118,0.000000,NaN,168
2,Australia,Sri Lanka,Melbourne,1,10,117,2.000000,NaN,168
3,Australia,Sri Lanka,Melbourne,3,10,116,4.500000,NaN,168
4,Australia,Sri Lanka,Melbourne,3,10,115,3.600000,NaN,168
...,...,...,...,...,...,...,...,...,...
76954,Sri Lanka,Australia,Colombo,125,2,3,6.410256,32.0,128
76955,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,128
76956,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,128
76957,Sri Lanka,Australia,Colombo,127,1,0,6.350000,33.0,128


In [26]:
 dt.dropna(inplace=True)

In [27]:
dt.isnull().sum()

batting_team           0
bowling_team           0
city                   0
current_score          0
wickets_left           0
balls_left             0
current_run_rate       0
last_five_over_runs    0
runs_x                 0
dtype: int64

In [28]:
dt.rename(columns={'runs_x':'score'},inplace=True)

In [29]:
dt=dt.sample(dt.shape[0])

In [30]:
dt

,batting_team,bowling_team,city,current_score,wickets_left,balls_left,current_run_rate,last_five_over_runs,score
47593,England,West Indies,London,43,7,66,4.777778,23.0,113
26778,Australia,England,Sydney,161,5,29,10.615385,38.0,221
6407,England,India,Bristol,140,6,36,10.000000,37.0,198
70232,New Zealand,Pakistan,Wellington,56,10,86,9.882353,50.0,196
38017,Afghanistan,India,St Lucia,69,7,41,5.240506,30.0,115
...,...,...,...,...,...,...,...,...,...
57967,West Indies,England,Barbados,64,9,74,8.347826,40.0,170
62803,Netherlands,New Zealand,Chittagong,89,7,39,6.592593,41.0,151
8516,Sri Lanka,India,Colombo,94,8,60,9.400000,43.0,152
75684,Bangladesh,Zimbabwe,Khulna,60,9,84,10.000000,53.0,180


In [31]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor
import xgboost
from xgboost import XGBRegressor

In [32]:
x=dt.drop('score',axis=1)
y=dt['score']

In [33]:
x

,batting_team,bowling_team,city,current_score,wickets_left,balls_left,current_run_rate,last_five_over_runs
47593,England,West Indies,London,43,7,66,4.777778,23.0
26778,Australia,England,Sydney,161,5,29,10.615385,38.0
6407,England,India,Bristol,140,6,36,10.000000,37.0
70232,New Zealand,Pakistan,Wellington,56,10,86,9.882353,50.0
38017,Afghanistan,India,St Lucia,69,7,41,5.240506,30.0
...,...,...,...,...,...,...,...,...
57967,West Indies,England,Barbados,64,9,74,8.347826,40.0
62803,Netherlands,New Zealand,Chittagong,89,7,39,6.592593,41.0
8516,Sri Lanka,India,Colombo,94,8,60,9.400000,43.0
75684,Bangladesh,Zimbabwe,Khulna,60,9,84,10.000000,53.0


In [34]:
y

47593    113
26778    221
6407     198
70232    196
38017    115
        ... 
57967    170
62803    151
8516     152
75684    180
75315    163
Name: score, Length: 58584, dtype: int64

In [35]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=10)

In [36]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(46867, 8)
(11717, 8)
(46867,)
(11717,)


In [37]:
lr1=ColumnTransformer([
    ('onehot',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

lr_pipe=Pipeline([
    ('lr1',lr1),
    ('lr2',StandardScaler()),
    ('lr3',LinearRegression())
])

lr_pipe.fit(xtrain,ytrain)

lr_ypred=lr_pipe.predict(xtest)

print('lr_r2score',r2_score(ytest,lr_ypred))
print('lr_mse',mean_squared_error(ytest,lr_ypred))
print('lr_rmse',np.sqrt(mean_squared_error(ytest,lr_ypred)))

lr_r2score 0.7154560795956971
lr_mse 311.98512372662555
lr_rmse 17.663100626068616


In [38]:
ri1=ColumnTransformer([
    ('onehot',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

ri_pipe=Pipeline([
    ('ri1',ri1),
    ('ri2',StandardScaler()),
    ('ri3',Ridge())
])

ri_pipe.fit(xtrain,ytrain)

ri_ypred=ri_pipe.predict(xtest)

print('lr_r2score',r2_score(ytest,ri_ypred))
print('lr_mse',mean_squared_error(ytest,ri_ypred))
print('lr_rmse',np.sqrt(mean_squared_error(ytest,ri_ypred)))

lr_r2score 0.7154626555039674
lr_mse 311.97791364266936
lr_rmse 17.662896524711606


In [39]:
xg1=ColumnTransformer([
    ('onehot',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

xg_pipe=Pipeline([
    ('xg1',xg1),
    ('xg2',StandardScaler()),
    ('xg3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

xg_pipe.fit(xtrain,ytrain)

xg_ypred=xg_pipe.predict(xtest)

print('xgb_r2score',r2_score(ytest,xg_ypred))
print('xgb_mse',mean_squared_error(ytest,xg_ypred))
print('xgb_rmse',np.sqrt(mean_squared_error(ytest,xg_ypred)))

xgb_r2score 0.9864506522099641
xgb_mse 14.856036778726775
xgb_rmse 3.8543529649899444


In [40]:
# pickle.dump(xg_pipe,open('t20.pkl','wb'))

In [41]:
pipe=pickle.load(open('t20.pkl','rb'))

In [42]:
dt.iloc[3987,:].values.reshape(1,-1)

array([['South Africa', 'Pakistan', 'Johannesburg', 83, 9, 49,
        7.014084507042254, 39.0, 188]], dtype=object)

In [43]:
inp=x.iloc[3987,:].values.reshape(1,-1)
inp

array([['South Africa', 'Pakistan', 'Johannesburg', 83, 9, 49,
        7.014084507042254, 39.0]], dtype=object)

In [45]:
int(pipe.predict(inp))

190